In [1]:
install.packages("forecast")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘colorspace’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’




In [11]:
install.packages("ggplot2")
library(ggplot2)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [8]:

library(forecast)

## 1. Read data ---------------------------------------------------------
df <- read.csv("western_area_SES.csv",
               stringsAsFactors = FALSE,
               check.names = FALSE)

# --- Diagnostic Step: Check column names and data structure ---
print("Column names of df:")
print(names(df))
print("Head of df:")
print(head(df))
# -------------------------------------------------------------

# Make sure data are in time order
df <- df[order(df$Period), ]

# Pick the coverage column
y <- df$Coverage
n <- length(y)

[1] "Column names of df:"
[1] "Period"   "Coverage"
[1] "Head of df:"
  Period Coverage
1 Jan-24   109.87
2 Feb-24   122.70
3 Mar-24   100.65
4 Apr-24    87.15
5 May-24    99.64
6 Jun-24    80.10


In [19]:
# Build a sequence of monthly dates from Jan 2024
dates <- seq(as.Date("2024-01-01"), by = "month", length.out = n)

hist_df <- data.frame(
  date     = dates,
  coverage = y
)

y_ts <- ts(y, start = c(2024, 1), frequency = 12)

ses_a03 <- HoltWinters(y_ts, alpha = 0.3, beta = FALSE, gamma = FALSE)
ses_a07 <- HoltWinters(y_ts, alpha = 0.7, beta = FALSE, gamma = FALSE)

fc_a03 <- forecast(ses_a03, h = 1)  # Dec 2025 forecast
fc_a07 <- forecast(ses_a07, h = 1)


#Build forecast dataframe

last_obs_date <- max(hist_df$date)
last_obs_val  <- tail(hist_df$coverage, 1)

# Next month
dec_2025_date <- seq(last_obs_date, by = "month", length.out = 2)[2]

fc_df <- data.frame(
  date     = c(dec_2025_date, dec_2025_date),
  alpha    = factor(c("α = 0.3", "α = 0.7"),
                    levels = c("α = 0.3", "α = 0.7")),
  forecast = c(as.numeric(fc_a03$mean),
               as.numeric(fc_a07$mean)),
  last_val = last_obs_val
)

fc_a03
fc_a07

         Point Forecast    Lo 80    Hi 80    Lo 95    Hi 95
Dec 2025       94.47152 78.25887 110.6842 69.67641 119.2666

         Point Forecast    Lo 80    Hi 80    Lo 95    Hi 95
Dec 2025       93.63762 79.00956 108.2657 71.26594 116.0093

In [9]:
## 2. Run SES using forecast package

alphas <- c(0.3, 0.7)
results <- list()

for (a in alphas) {
  # Fit SES model with specified alpha
  fit <- ses(y, h = 1, alpha = a, initial = "simple")

  f_col <- paste0("Forecast_alpha_", gsub("\\.", "_", a))
  e_col <- paste0("Error_alpha_",   gsub("\\.", "_", a))

  # Store fitted values (in-sample forecasts)
  df[[f_col]] <- fitted(fit)

  # Calculate errors: Actual - Forecast
  df[[e_col]] <- y - df[[f_col]]

  # Store December 2025 forecast
  results[[as.character(a)]] <- as.numeric(fit$mean[1])
}

## 3. Error metrics for last 6 months (Jun–Nov 2025)

last6_idx <- (n - 5):n

for (a in alphas) {
  e_col <- paste0("Error_alpha_", gsub("\\.", "_", a))
  err6  <- df[[e_col]][last6_idx]

  MAE <- mean(abs(err6))
  MSE <- mean(err6^2)
  RMSE <- sqrt(MSE)

  cat("\nAlpha =", a, "\n")
  cat("  Forecast for Dec 2025:", round(results[[as.character(a)]], 2), "%\n")
  cat("  MAE (last 6 months):  ", round(MAE, 3), "\n")
  cat("  MSE (last 6 months):  ", round(MSE, 3), "\n")
  cat("  RMSE (last 6 months): ", round(RMSE, 3), "\n")
}

## 4. Display last 6 months results
cat("\n\nLast 6 months comparison:\n")
print(tail(df[, c("Period",
                  "Coverage",
                  grep("Forecast_alpha_", names(df), value = TRUE),
                  grep("Error_alpha_",   names(df), value = TRUE))], 6))


Alpha = 0.3 
  Forecast for Dec 2025: 94.47 %
  MAE (last 6 months):   6.44 
  MSE (last 6 months):   55.649 
  RMSE (last 6 months):  7.46 

Alpha = 0.7 
  Forecast for Dec 2025: 93.64 %
  MAE (last 6 months):   7.264 
  MSE (last 6 months):   84.43 
  RMSE (last 6 months):  9.189 


Last 6 months comparison:
   Period Coverage Forecast_alpha_0_3 Forecast_alpha_0_7 Error_alpha_0_3
11 Nov-24    87.13           95.12623           98.71367      -7.9962341
23 Nov-25   103.45           92.72736           90.60510      10.7226362
10 Oct-24   103.45           95.94415           99.59653       7.5058453
22 Oct-25    88.46           98.19591          102.29396      -9.7359083
9  Sep-24    93.57           95.27514           92.61019      -1.7051358
21 Sep-25    93.79           94.76360           93.28206      -0.9735951
   Error_alpha_0_7
11     -11.5836748
23      12.8448976
10       3.8534693
22     -13.8339592
9        0.9598122
21       0.5079437
